# Coursera Capstone Course Week5 Assignment
This project is to find a location in Boston for a new Chinese restaurant. It will leverage the neighborhood data from wikipedia and the location data from FourSquare. The codes for clustering the neighborhoods for completing this project is listed following.

### Import the libraries

In [1]:
# Import the libraries

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0          conda-forge
    geopy:           

### Get the required neighborhoods

In [2]:
# Get the required neighborhoods data from wikipedia

from bs4 import BeautifulSoup
import urllib.request

soup = BeautifulSoup(urllib.request.urlopen("https://en.wikipedia.org/wiki/Neighborhoods_in_Boston"),
    "html5lib")

l = soup.select("#mw-content-text ul")[0].select("li a")
neighborhoodlist = []

for n in l:
    neighborhoodlist.append(n["title"].split(",")[0])

print(neighborhoodlist)

['Allston', 'Back Bay', 'Bay Village', 'Beacon Hill', 'Brighton', 'Charlestown', 'Chinatown', 'Leather District', 'Dorchester', 'Downtown Boston', 'East Boston', 'Fenway-Kenmore', 'Longwood', 'Hyde Park', 'Jamaica Plain', 'Mattapan', 'Mission Hill', 'North End', 'Roslindale', 'Roxbury', 'South Boston', 'South End', 'West End', 'West Roxbury']


In [3]:
# create a dataframe consisting of the neihorhoodlist

bos_df = pd.DataFrame({"Neighborhood": neighborhoodlist})

bos_df.head()

,Neighborhood
0,Allston
1,Back Bay
2,Bay Village
3,Beacon Hill
4,Brighton


In [4]:
# check the shape of the dataframe

bos_df.shape

(24, 1)

### Get the coordinates of the neighborhoods

In [5]:
# Use Allston as an example to get the geographical corrdinates

locator = Nominatim(user_agent= "myGeocoder")
location = locator.geocode("Allston, Boston, United States")
lat = location.latitude
lon = location.longitude
print('The geograpical coordinate of Allston are {}, {}.'.format(lat, lon))

The geograpical coordinate of Allston are 42.3554344, -71.1321271.


In [6]:
# Create a list for all the neighborhoods
addresslist = []
for n in neighborhoodlist:
    n = n + ", Boston, United States"
    addresslist.append(n)

print(addresslist)

['Allston, Boston, United States', 'Back Bay, Boston, United States', 'Bay Village, Boston, United States', 'Beacon Hill, Boston, United States', 'Brighton, Boston, United States', 'Charlestown, Boston, United States', 'Chinatown, Boston, United States', 'Leather District, Boston, United States', 'Dorchester, Boston, United States', 'Downtown Boston, Boston, United States', 'East Boston, Boston, United States', 'Fenway-Kenmore, Boston, United States', 'Longwood, Boston, United States', 'Hyde Park, Boston, United States', 'Jamaica Plain, Boston, United States', 'Mattapan, Boston, United States', 'Mission Hill, Boston, United States', 'North End, Boston, United States', 'Roslindale, Boston, United States', 'Roxbury, Boston, United States', 'South Boston, Boston, United States', 'South End, Boston, United States', 'West End, Boston, United States', 'West Roxbury, Boston, United States']


In [7]:
#Get all the geographic coordinates of the neighborhoods in botston

lat_list = []
lon_list = []

for address in addresslist:
    locator = Nominatim(user_agent = "myGeocoder")
    location = locator.geocode(address)
    if location:
        latitude = location.latitude
        longitude = location.longitude
        lat_list.append(latitude)
        lon_list.append(longitude)
    else:
        lat_list.append("No Data")
        lon_list.append("No Data")

print(lat_list)
print(lon_list)

[42.3554344, 42.3507067, 42.35001105, 42.3587085, 42.3500971, 42.3778749, 42.3494053, 'No Data', 42.2973205, 42.3554309, 42.3750973, 'No Data', 42.3417184, 42.2556543, 42.3098201, 42.2675657, 'No Data', 42.3650974, 42.2912093, 42.3248426, 42.3334312, 42.34131, 42.3639186, 42.2792649]
[-71.1321271, -71.0797297, -71.0669477958571, -71.067829, -71.1564423, -71.0619957, -71.0635823927835, 'No Data', -71.0744952, -71.0605001, -71.0392173, 'No Data', -71.109922, -71.1244963, -71.1203299, -71.0924273, 'No Data', -71.0544954, -71.1244966, -71.0950158, -71.0494949, -71.0772298, -71.0638993, -71.1494972]


In [8]:
# create a new dataframe

bos_df = pd.DataFrame({"Neighborhood": neighborhoodlist, "Latitude": lat_list, "Longitude": lon_list})
bos_df

,Neighborhood,Latitude,Longitude
0,Allston,42.3554,-71.1321
1,Back Bay,42.3507,-71.0797
2,Bay Village,42.35,-71.0669
3,Beacon Hill,42.3587,-71.0678
4,Brighton,42.3501,-71.1564
5,Charlestown,42.3779,-71.062
6,Chinatown,42.3494,-71.0636
7,Leather District,No Data,No Data
8,Dorchester,42.2973,-71.0745
9,Downtown Boston,42.3554,-71.0605


##### Because Nominatim couldn't find the geographical coordinates for Leather District, Fenway-kenmore, and Mission Hill, I did it manually. For the neighborhood Leather District, the coordinates are 42.3505° N, 71.0579° W. For the neighborhood Fenway-Kenmore, the coordinates are 42.3429° N, 71.1003° W. And for the neighborhood Mission Hill, the coordinates are 42.3296° N, 71.1062° W.

In [9]:
# Manually change the value of the Latitude and Longitude of the 3 neighborhoods

bos_df.loc[7]["Latitude"] = 42.3505
bos_df.loc[7]["Longitude"] = -71.0579
bos_df.loc[11]["Latitude"] = 42.3429
bos_df.loc[11]["Longitude"] = -71.1003
bos_df.loc[16]["Latitude"] = 42.3296
bos_df.loc[16]["Longitude"] = -71.1062

In [10]:
# show the dataframe

bos_df

,Neighborhood,Latitude,Longitude
0,Allston,42.3554,-71.1321
1,Back Bay,42.3507,-71.0797
2,Bay Village,42.35,-71.0669
3,Beacon Hill,42.3587,-71.0678
4,Brighton,42.3501,-71.1564
5,Charlestown,42.3779,-71.062
6,Chinatown,42.3494,-71.0636
7,Leather District,42.3505,-71.0579
8,Dorchester,42.2973,-71.0745
9,Downtown Boston,42.3554,-71.0605


### Visualize Boston neighborhoods

In [12]:
# create a map of Boston using latitude and longitude values

map_boston = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map

for lat, lng, label in zip(bos_df['Latitude'], bos_df['Longitude'], bos_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_boston)

map_boston

### Use FourSqare API to explore the neighborhoods in Boston

In [13]:
CLIENT_ID = 'SO5KPPMTWRN01UJHEZ1YVPEBKGPHBUWUYJD2WH4X5IYI1G43' # your Foursquare ID
CLIENT_SECRET = '42PRFWEQ01GO4KZ4LPWUAVTZDIJ1WA5BFMVOT2FQXAL5KBTF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SO5KPPMTWRN01UJHEZ1YVPEBKGPHBUWUYJD2WH4X5IYI1G43
CLIENT_SECRET:42PRFWEQ01GO4KZ4LPWUAVTZDIJ1WA5BFMVOT2FQXAL5KBTF


In [14]:
# Get the top 100 venuews in the neighborhoods within a radius of 500 meters

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [15]:
# Create a function to get the venues from all the neighborhoods

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
# write the code to run the above funtion on each neighborhood and create a new dataframe called venues_df

venues_df = getNearbyVenues(names=bos_df['Neighborhood'],
                            latitudes=bos_df['Latitude'],
                            longitudes=bos_df['Longitude']
                            )

print(venues_df.shape)
venues_df.head()

Allston
Back Bay
Bay Village
Beacon Hill
Brighton
Charlestown
Chinatown
Leather District
Dorchester
Downtown Boston
East Boston
Fenway-Kenmore
Longwood
Hyde Park
Jamaica Plain
Mattapan
Mission Hill
North End
Roslindale
Roxbury
South Boston
South End
West End
West Roxbury
(1101, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Allston,42.355434,-71.132127,Lulu's Allston,42.355068,-71.134107,Comfort Food Restaurant
1,Allston,42.355434,-71.132127,Kaju Tofu House,42.354329,-71.132374,Korean Restaurant
2,Allston,42.355434,-71.132127,Fish Market Sushi Bar,42.353039,-71.132975,Sushi Restaurant
3,Allston,42.355434,-71.132127,Azama Grill,42.354422,-71.132358,Falafel Restaurant
4,Allston,42.355434,-71.132127,Allston Diner,42.354979,-71.134295,Diner


In [17]:
# check how many venues were returned for each neighborhood

venues_df.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allston,78,78,78,78,78,78
Back Bay,100,100,100,100,100,100
Bay Village,59,59,59,59,59,59
Beacon Hill,42,42,42,42,42,42
Brighton,46,46,46,46,46,46
Charlestown,34,34,34,34,34,34
Chinatown,78,78,78,78,78,78
Dorchester,11,11,11,11,11,11
Downtown Boston,100,100,100,100,100,100


In [18]:
# find out how many unique categories can be curated from all the returned venues

print('There are {} uniques categories.'.format(len(venues_df['Venue Category'].unique())))

There are 201 uniques categories.


### Analyze each neighborhood

In [19]:
# one hot encoding
bos_onehot = pd.get_dummies(venues_df[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bos_onehot['Neighborhood'] = venues_df['Neighborhood']

# move neighborhood column to the first column
col_name = "Neighborhood"
first_col = bos_onehot.pop(col_name)
bos_onehot.insert(0, col_name, first_col)

bos_onehot.head()

,Neighborhood,ATM,Accessories Store,American Restaurant,Arepa Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Garden,Belgian Restaurant,Big Box Store,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Café,Caribbean Restaurant,Cemetery,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Food,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Historic Site,History Museum,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Library,Liquor Store,Lounge,Malay Restaurant,Market,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,National Park,New American Restaurant,Noodle House,Opera House,Optical Shop,Other Repair Shop,Outdoor Sculpture,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Record Shop,Rental Car Location,Resort,Restaurant,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Ski Chalet,Smoke Shop,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tiki Bar,Tour Provider,Tourist Information Center,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Allston,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Allston,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Allston,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0

In [20]:
# check the size of the new dataframe

bos_onehot.shape

(1101, 201)

### Group rows by neighborhood and by taking the mean of the frequency of occureence of each category

In [21]:
bos_grouped = bos_onehot.groupby("Neighborhood").mean().reset_index()
print(bos_grouped.shape)
bos_grouped

(24, 201)


,Neighborhood,ATM,Accessories Store,American Restaurant,Arepa Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Garden,Belgian Restaurant,Big Box Store,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Café,Caribbean Restaurant,Cemetery,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Food,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Historic Site,History Museum,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Library,Liquor Store,Lounge,Malay Restaurant,Market,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,National Park,New American Restaurant,Noodle House,Opera House,Optical Shop,Other Repair Shop,Outdoor Sculpture,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Record Shop,Rental Car Location,Resort,Restaurant,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Ski Chalet,Smoke Shop,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tiki Bar,Tour Provider,Tourist Information Center,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Allston,0.000000,0.000000,0.000000,0.000000,0.012821,0.000000,0.025641,0.000000,0.012821,0.000000,0.000000,0.038462,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.012821,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.025641,0.000000,0.012821,0.000000,0.000000,0.012821,0.000000,0.000000,0.000000,0.038462,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.012821,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.012821,0.000000,0.000000,0.000000,0.000000,0.012821,0.000000,0.025641,0.000000,0.000000,0.012821,0.000000,0.012821,0.012821,0.000000,0.012821,0.000000,0.012821,0.000000,0.000000,0.000000,0.000000,0.025641,0.012821,0.000000,0.000000,0.000000,0.000000,0.012821,0.000000,0.000000,0.000000,0.000000,0.000000,0.012821,0.025641,0.000000,0.012821,0.000000,0.000000,0.000000,0.0,0.012821,0.000000,0.012821,0.000000,0.000000,0.000000,0.012821,0.012821,0.000000,0.025641,0.012821,0.000000,0.00,0.000000,0.000000,0.00000,0.076923,0.00000,0.012821,0.0,0.000000,0.012821,0.000000,0.0000,0.000000,0.012821,0.000000,0.025641,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012821,0.000000,0.000000,0.00,0.000000,0.00,

In [22]:
# print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in bos_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = bos_grouped[bos_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Allston----
                    venue  freq
0       Korean Restaurant  0.08
1         Thai Restaurant  0.04
2             Pizza Place  0.04
3      Chinese Restaurant  0.04
4  Thrift / Vintage Store  0.04


----Back Bay----
                 venue  freq
0          Coffee Shop  0.09
1   Italian Restaurant  0.06
2  American Restaurant  0.05
3                Hotel  0.05
4       Clothing Store  0.04


----Bay Village----
                   venue  freq
0                  Hotel  0.07
1         Sandwich Place  0.07
2     Italian Restaurant  0.05
3  Performing Arts Venue  0.05
4     Seafood Restaurant  0.05


----Beacon Hill----
                 venue  freq
0          Pizza Place  0.07
1   Italian Restaurant  0.07
2            Hotel Bar  0.05
3  American Restaurant  0.05
4    French Restaurant  0.05


----Brighton----
         venue  freq
0  Bus Station  0.07
1       Bakery  0.07
2  Pizza Place  0.07
3         Bank  0.07
4          Pub  0.04


----Charlestown----
               venue  freq
0

In [24]:
# put the above data into a pandas dataframe

# write a function to sort the venues in descending order

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [26]:
# create the new dataframe and display the top 10 venues for each neighborhood

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = bos_grouped['Neighborhood']

for ind in np.arange(bos_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bos_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allston,Korean Restaurant,Pizza Place,Thai Restaurant,Bakery,Vegetarian / Vegan Restaurant,Chinese Restaurant,Thrift / Vintage Store,Fried Chicken Joint,Pharmacy,Bubble Tea Shop
1,Back Bay,Coffee Shop,Italian Restaurant,American Restaurant,Hotel,Seafood Restaurant,Clothing Store,Ice Cream Shop,Gym / Fitness Center,Juice Bar,Shopping Mall
2,Bay Village,Hotel,Sandwich Place,Italian Restaurant,Performing Arts Venue,Seafood Restaurant,Bakery,American Restaurant,Theater,Spa,Park
3,Beacon Hill,Pizza Place,Italian Restaurant,American Restaurant,Museum,French Restaurant,Restaurant,Hotel Bar,Yoga Studio,Hotpot Restaurant,Park
4,Brighton,Pizza Place,Bank,Bus Station,Bakery,Coffee Shop,Chinese Restaurant,Grocery Store,Pub,Deli / Bodega,Mobile Phone Shop


### Cluster neighborhoods
#### Run k-means to cluster the neighborhoods into 5 clusters

In [27]:
# set number of clusters
kclusters = 5

bos_grouped_clustering = bos_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bos_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 3, 0, 0], dtype=int32)

### Create a dataframe which inclueds the clusters as well as the top 10 venues for each neighborhood

In [30]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
bos_merged = bos_df
bos_merged = bos_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

# check the last columns
bos_merged.head() 

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allston,42.3554,-71.1321,0,Korean Restaurant,Pizza Place,Thai Restaurant,Bakery,Vegetarian / Vegan Restaurant,Chinese Restaurant,Thrift / Vintage Store,Fried Chicken Joint,Pharmacy,Bubble Tea Shop
1,Back Bay,42.3507,-71.0797,0,Coffee Shop,Italian Restaurant,American Restaurant,Hotel,Seafood Restaurant,Clothing Store,Ice Cream Shop,Gym / Fitness Center,Juice Bar,Shopping Mall
2,Bay Village,42.35,-71.0669,0,Hotel,Sandwich Place,Italian Restaurant,Performing Arts Venue,Seafood Restaurant,Bakery,American Restaurant,Theater,Spa,Park
3,Beacon Hill,42.3587,-71.0678,0,Pizza Place,Italian Restaurant,American Restaurant,Museum,French Restaurant,Restaurant,Hotel Bar,Yoga Studio,Hotpot Restaurant,Park
4,Brighton,42.3501,-71.1564,0,Pizza Place,Bank,Bus Station,Bakery,Coffee Shop,Chinese Restaurant,Grocery Store,Pub,Deli / Bodega,Mobile Phone Shop


### Visualize the resulting clusters

In [31]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bos_merged['Latitude'], bos_merged['Longitude'], bos_merged['Neighborhood'], bos_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine clusters

In [33]:
# examine cluster 1

bos_merged.loc[bos_merged['Cluster Labels'] == 0, bos_merged.columns[[0] + list(range(4, bos_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allston,Korean Restaurant,Pizza Place,Thai Restaurant,Bakery,Vegetarian / Vegan Restaurant,Chinese Restaurant,Thrift / Vintage Store,Fried Chicken Joint,Pharmacy,Bubble Tea Shop
1,Back Bay,Coffee Shop,Italian Restaurant,American Restaurant,Hotel,Seafood Restaurant,Clothing Store,Ice Cream Shop,Gym / Fitness Center,Juice Bar,Shopping Mall
2,Bay Village,Hotel,Sandwich Place,Italian Restaurant,Performing Arts Venue,Seafood Restaurant,Bakery,American Restaurant,Theater,Spa,Park
3,Beacon Hill,Pizza Place,Italian Restaurant,American Restaurant,Museum,French Restaurant,Restaurant,Hotel Bar,Yoga Studio,Hotpot Restaurant,Park
4,Brighton,Pizza Place,Bank,Bus Station,Bakery,Coffee Shop,Chinese Restaurant,Grocery Store,Pub,Deli / Bodega,Mobile Phone Shop
5,Charlestown,Pizza Place,Pub,Gastropub,Donut Shop,Coffee Shop,Convenience Store,Yoga Studio,National Park,Plaza,Pharmacy
6,Chinatown,Chinese Restaurant,Bakery,Asian Restaurant,Seafood Restaurant,Bubble Tea Shop,Performing Arts Venue,Hotel,Restaurant,Coffee Shop,Sandwich Place
7,Leather District,Chinese Restaurant,Asian Restaurant,Bakery,Coffee Shop,Sandwich Place,Sushi Restaurant,American Restaurant,Food Truck,Dessert Shop,Noodle House
9,Downtown Boston,Coffee Shop,Chinese Restaurant,Park,Bakery,Historic Site,Sushi Restaurant,Burger Joint,Asian Restaurant,Hotel,New American Restaurant
10,East Boston,Latin American Restaurant,Convenience Store,Liquor Store,Art Gallery,Pharmacy,Pizza Place,Sandwich Place,Colombian Restaurant,Café,Fast Food Restaurant


In [34]:
# examine cluster 2

bos_merged.loc[bos_merged['Cluster Labels'] == 1, bos_merged.columns[[0] + list(range(4, bos_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,West Roxbury,Home Service,Lawyer,Yoga Studio,Doctor's Office,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Electronics Store,Dumpling Restaurant


In [35]:
# examine cluster 3

bos_merged.loc[bos_merged['Cluster Labels'] == 2, bos_merged.columns[[0] + list(range(4, bos_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Mattapan,Yoga Studio,Bakery,Fast Food Restaurant,Mobile Phone Shop,Nail Salon,Pharmacy,Caribbean Restaurant,Southern / Soul Food Restaurant,Convenience Store,Automotive Shop


In [36]:
# examine cluster 4

bos_merged.loc[bos_merged['Cluster Labels'] == 3, bos_merged.columns[[0] + list(range(4, bos_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Dorchester,Food,Market,Breakfast Spot,Fried Chicken Joint,Plaza,Gym,Café,Pizza Place,Vegetarian / Vegan Restaurant,Southern / Soul Food Restaurant


In [37]:
# examine cluster 5

bos_merged.loc[bos_merged['Cluster Labels'] == 4, bos_merged.columns[[0] + list(range(4, bos_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Roxbury,Park,Metro Station,Plaza,Art Gallery,Gym,Furniture / Home Store,Doctor's Office,Farmers Market,Falafel Restaurant,Event Space


## Observation:

Most of the neighborhoods are located in cluster 1 where cafe, restaurants, stores are abundant. While the places with high frequency in cluster 2,3,4, and 5 are different. In cluster 2, people visited those places mainly for getting services. In cluster 3, the places that people visit frequently cover a wide range of types, and it can be seen that it is a neighborhood with relatively convenient life. In cluster 4, although lots of places mentioned here are about food, they are mainly related to fast food. In cluster 5, the location people frequently visit are park, metro, plaza, art gallery, gym, and so on. It can be seen that this is a good neighborhood for entertaining.
In a word, if I need to find a cluster for openning a new Chinese restaurant, it definitely should be in cluster 1. However, I still need to compare different neighborhoods in cluster 1 to find a more specific location.